In [ ]:
import requests
from bs4 import BeautifulSoup
import time

URL = "https://www.emploi-territorial.fr/emploi-mobilite/?adv-search=Data&search-ville=69123"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
def scrape_offers_requests(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    offers = []

    # chaque offre = une ligne <tr id="O....">
    for row in soup.select("tr[id^='O']"):

        link = row.select_one("a.lien-details-offre")
        if not link:
            continue

        offer_title = link.get_text(strip=True)
        offer_url = "https://www.emploi-territorial.fr" + link["href"]

        offers.append({
            "title": offer_title,
            "url": offer_url
        })

    return offers


offers = scrape_offers_requests(URL)
print(f"{len(offers)} offres trouvées via requests")
for o in offers:
    print(o)

20 offres trouvées via requests
{'title': 'RESPONSABLE DU PÔLE APPLICATIFS ET DATA  (F/H)', 'url': 'https://www.emploi-territorial.fr/offre/o069251205001174-responsable-pOle-applicatifs-data-h'}
{'title': 'CHEF DU SERVICE CONTRATS, PROCESS & DATA  (F/H)', 'url': 'https://www.emploi-territorial.fr/offre/o069251208000412-chef-service-contrats-process-data-h'}
{'title': 'CHEF DE PROJET DONNÉES (F/H)', 'url': 'https://www.emploi-territorial.fr/offre/o069251205001563-chef-projet-donnEes-h'}
{'title': "JURISTE PROTECTION DES DONNÉES / TECHNOLOGIE DE L'INFORMATION / PROPRIÉTÉ INTELLECTUELLE (F/H)", 'url': 'https://www.emploi-territorial.fr/offre/o069251209000757-juriste-protection-donnEes-technologie-information-propriEtE-intellectuelle-h'}
{'title': 'Développeur économique Portes du Sud (H/F)', 'url': 'https://www.emploi-territorial.fr/offre/o069251128000322-developpeur-economique-portes-sud'}
{'title': 'Assistant administratif Petite Enfance', 'url': 'https://www.emploi-territorial.fr/offre

In [26]:
def scrape_offer_detail(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Conteneur principal du détail de l'offre
    main = soup.select_one("div#contenuOffre, div.contenu-offre, main")

    if not main:
        return ""

    # Supprimer éléments inutiles
    for tag in main(["script", "style", "button", "nav"]):
        tag.decompose()

    text = main.get_text(separator="\n", strip=True)
    return text


In [ ]:
URL = "https://www.emploi-territorial.fr/emploi-mobilite/?adv-search=Data&search-ville=69123"

offers = scrape_offers_requests(URL)

corpus = []

for offer in offers:
    print(f"Scraping : {offer['title']}")
    full_text = scrape_offer_detail(offer["url"])

    corpus.append({
        "title": offer["title"],
        "url": offer["url"],
        "text": full_text
    })

    time.sleep(1)  # pour éviter les surchages même si on récupère pas 10K emplois d'un coup



Scraping : RESPONSABLE DU PÔLE APPLICATIFS ET DATA  (F/H)
Scraping : CHEF DU SERVICE CONTRATS, PROCESS & DATA  (F/H)
Scraping : CHEF DE PROJET DONNÉES (F/H)
Scraping : JURISTE PROTECTION DES DONNÉES / TECHNOLOGIE DE L'INFORMATION / PROPRIÉTÉ INTELLECTUELLE (F/H)
Scraping : Développeur économique Portes du Sud (H/F)
Scraping : Assistant administratif Petite Enfance
Scraping : RESPONSABLE DU PÔLE PERFORMANCE DE LA PRODUCTION ET QUALITÉ DE SERVICE (F/H)
Scraping : Gestionnaire comptable du service dépenses
Scraping : Responsable du service masse salariale et effectifs (H/F)
Scraping : Directeur de crèche familiale
Scraping : Agent d'urbanisme, enseignes commerces et gestion de l'espace public
Scraping : Agent d'urbanisme, enseignes commerces et gestion de l'espace public
Scraping : CHARGE DE MISSION EQUIPE MOBILE
Scraping : Agent.e de gestion administrative et financière – CDD 6 mois
Scraping : Chargé de mission action foncière (h/f)
Scraping : Maraîcher (h/f)
Scraping : Diététicien.ne
Sc

In [ ]:
import csv

def save_corpus_to_csv(corpus, filename="corpus_offres_data.csv"):
    with open(filename, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["title", "url", "text"],
            delimiter=";"
        )
        writer.writeheader()
        for row in corpus:
            writer.writerow(row)
save_corpus_to_csv(corpus)

# petite sauvegarde en csv rapide pour pas le perdre (optionnelle pour la suite)
